In [1]:
from transformers import BitsAndBytesConfig
from llama_index.core import PromptTemplate, StorageContext, load_index_from_storage, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.response.notebook_utils import display_response

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

model_name = config['indexing']['indexing_model']
context_window = int(config['indexing']['context_window'])
max_new_tokens = int(config['indexing']['max_new_tokens'])
temp = float(config['indexing']['temperature'])
top_k = int(config['indexing']['top_k'])
top_p = float(config['indexing']['top_p'])
embed_model = config['indexing']['embed_model']

In [3]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)

labels = ["Regulación y Legislación", "Administración Pública y Procedimientos", "Educación y Cultura"] 

system_prompt = """
    Actúa como un asistente especializado en consultas sobre documentos del Boletín Oficial del Estado (BOE). Tu tarea es 
    proporcionar información relevante y precisa basándote únicamente en los documentos disponibles del BOE. Es fundamental 
    que sigas estas instrucciones para asegurar la precisión y relevancia de tus respuestas:

    1. Responde únicamente en el idioma en el que se formule la pregunta. Esto es crucial, ya que las personas a las que asistes 
       solo entienden el idioma en el que preguntan.
    2. Proporciona respuestas detalladas y específicas, utilizando solo la información contenida en los documentos del BOE. 
       No hagas suposiciones ni especulaciones.
    3. Si la información solicitada no está disponible en los documentos del BOE, informa al usuario de manera clara y directa 
       que no tienes la información requerida.
    4. Incluye referencias al documento o documentos relevantes del BOE siempre que sea posible para respaldar tus respuestas.
    5. Utiliza la información más actualizada disponible en los documentos del BOE para responder a las preguntas.

    Aquí hay algunos ejemplos de cómo podrías responder:

    Ejemplo 1:
    Pregunta: ¿Cuál es la última modificación de la Ley de Propiedad Intelectual?
    Respuesta: Según el documento del BOE del [fecha], la última modificación de la Ley de Propiedad Intelectual se realizó el [fecha de modificación], donde se introdujeron cambios en [detalles específicos de los cambios]. Puedes consultar el documento completo en el BOE en la sección [sección del BOE].

    Ejemplo 2:
    Pregunta: ¿Qué establece el Real Decreto 123/2023 sobre el teletrabajo?
    Respuesta: El Real Decreto 123/2023, publicado en el BOE el [fecha], establece las condiciones y requisitos para el teletrabajo, incluyendo [detalles específicos]. Para más información, puedes revisar el documento en la sección [sección del BOE].

    Si no tienes la información requerida, responde de la siguiente manera:
    "Lo siento, no tengo información sobre eso en los documentos disponibles del BOE. Por favor, revisa el BOE directamente o proporciona más detalles para una búsqueda más precisa."

    Siguiendo estas pautas, asegúrate de que cada respuesta sea clara, precisa y útil para el usuario.
"""

query_wrapper = PromptTemplate(
f"<s> [INST] {system_prompt} Dada la siguiente pregunta del usuario, proporciona una respuesta detallada y específica utilizando la información contenida en los documentos. "
"Asegúrate de incluir referencias al documento o documentos relevantes cuando sea posible.\nPregunta: {query_str} [/INST] </s>\n"
)

llm = HuggingFaceLLM(                                                        
    model_name = model_name,                                                 
    tokenizer_name = model_name,                                             
    query_wrapper_prompt = query_wrapper,                            
    context_window = context_window,                                            
    max_new_tokens = max_new_tokens,                                           
    model_kwargs = {"quantization_config": quantization_config},      
    generate_kwargs = {"do_sample": True, "temperature": temp, "top_k": top_k, "top_p": top_p}, 
    device_map = "auto",                                              
    system_prompt = system_prompt
)


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
service_context = ServiceContext.from_defaults(llm = llm, embed_model=embed_model)

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8140\2540478816.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm = llm, embed_model=embed_model)


In [5]:
vector_store_leg = FaissVectorStore.from_persist_dir("indexes/legislacion_y_regulacion")
storage_context_leg = StorageContext.from_defaults(vector_store=vector_store_leg, persist_dir="indexes/legislacion_y_regulacion")
index_leg = load_index_from_storage(storage_context=storage_context_leg, service_context = service_context)

qe_leg = index_leg.as_query_engine(llm=llm, response_mode="compact")

In [6]:
vector_store_edu = FaissVectorStore.from_persist_dir("indexes/educacion_y_cultura")
storage_context_edu = StorageContext.from_defaults(vector_store=vector_store_edu, persist_dir="indexes/educacion_y_cultura")
index_edu = load_index_from_storage(storage_context=storage_context_edu, service_context = service_context)

qe_edu = index_edu.as_query_engine(llm=llm, response_mode="compact")

In [7]:
vector_store_adm = FaissVectorStore.from_persist_dir("indexes/administracion_publica_y_procedimientos")
storage_context_adm = StorageContext.from_defaults(vector_store=vector_store_adm, persist_dir="indexes/administracion_publica_y_procedimientos")
index_adm = load_index_from_storage(storage_context=storage_context_adm, service_context = service_context)

qe_adm = index_adm.as_query_engine(llm=llm, response_mode="compact")

### Preguntas sobre legislación y regulación

In [8]:
response = qe_leg.query("¿Cuál es el propósito principal del Decreto Legislativo 3/2000 del Gobierno de Aragón?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
c:\Program Files\Python312\Lib\site-packages\transformers\models\mixtral\modeling_mixtral.py:765: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** El Decreto Legislativo 3/2000 del Gobierno de Aragón tiene como propósito principal modificar la estructura del Gobierno de Aragón, específicamente introduciendo la posibilidad de crear la figura del Vicepresidente, quien estará vinculado al desempeño de una Consejería y será nombrado por el Presidente del Gobierno de Aragón. Esta modificación es coherente con la novedad normativa y conlleva pequeños cambios en la regulación del Presidente en funciones, la delegación de funciones del Presidente, las Comisiones Delegadas del Gobierno, el sistema de incompatibilidades y la composición del Gobierno. El objetivo de estas modificaciones es mejorar la eficacia del funcionamiento de la Administración que el Gobierno de Aragón dirige, lo que es justificable debido a la evolución desacompasada de las estructuras de Gobierno y Administración en relación a la Ley 3/1984, de 22 de junio, del Presidente, de la Diputación General y de la Administración de la Comunidad Autónoma.

Referencia:
<https://www.boe.es/buscar/doc.php?id=BOA-d-2010-90041>

In [9]:
response = qe_leg.query("¿Qué establece la última reforma del Código Penal respecto al delito de fraude online?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La última reforma del Código Penal no especifica directamente el delito de fraude online en el documento del Boletín Oficial del Estado (BOE) correspondiente a la Ley 11/2021, de 9 de noviembre. Sin embargo, la reforma introduce importantes modificaciones en la regulación del delito fiscal, que pueden estar relacionadas con el fraude online.

Entre estas modificaciones, se encuentra la regulación explícita de la elusión del pago de retenciones e ingresos a cuenta de retribuciones en especie como elemento integrante del tipo delictivo. Esto es especialmente relevante, ya que estos ilícitos tienen una gran importancia y trascendencia en el ordenamiento tributario.

Además, se determina la relación entre la regularización tributaria y el delito fiscal, lo que conlleva la ausencia de responsabilidad penal para estas conductas siempre que se realicen de manera espontánea. Esta regulación se hace necesaria para garantizar el cumplimiento voluntario de las obligaciones tributarias por los contribuyentes y para integrar plenamente las regularizaciones autorizadas en el conjunto del ordenamiento jurídico.

Por otra parte, la reforma actualiza las cuantías determinantes de la comisión del delito de fraude en subvenciones y del delito contable, y crea una figura de delito contra la Seguridad Social, similar al delito fiscal, con el objetivo de proteger adecuadamente su patrimonio y garantizar el cumplimiento de sus funciones institucionales.

Sin embargo, ninguno de estos documentos hace referencia específica al delito de fraude online. Por lo tanto, no puedo proporcionar una respuesta detallada y específica a la pregunta, ya que la información relevante no está disponible en los documentos proporcionados.

References:
- BOE-A-2021-17307: Ley 11/2021, de 9 de noviembre, por la que se modifica la Ley 58/2003, de 17 de diciembre, General Tributaria, para la mejora de la recaudación en el ámbito de las competencias de la Agencia Estatal de Administración Tributaria.

In [10]:
response = qe_leg.query("What does the latest reform of the Penal Code establish regarding the crime of online fraud?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Based on the information provided, I have analyzed the Boletín Oficial del Estado (BOE) documents and found that there is no specific reference to the latest reform of the Penal Code regarding the crime of online fraud. However, I can share some general insights about the process of Penal Code reforms in Spain.

The Penal Code in Spain can be modified through organic laws, which must follow a specific procedure involving different readings and votes in the Congress and the Senate. These modifications usually aim to adapt the code to new social contexts, new forms of criminal behavior, and technological advancements.

In the case of online fraud, it is possible that the latest reform of the Penal Code includes provisions to address this issue, but the specific details of such provisions are not available in the documents you provided. I recommend consulting the most recent version of the Penal Code or official government sources for the most accurate and up-to-date information on this topic.

For your reference, here are the documents I used to form my answer:
1. https://www.boe.es/buscar/doc.php?id=BOE-A-2015-6464
2. https://www.boe.es/buscar/doc.php?id=BOE-A-1988-12908

These documents do not contain specific information about the latest reform of the Penal Code regarding online fraud, but they can still provide valuable context about the Penal Code reform process and the legal framework in Spain.

In [11]:
response = qe_leg.query("What is the main purpose of Legislative Decree 3/2000 of the Government of Aragon?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La pregunta se refiere al Decreto Legislativo 3/2000 del Gobierno de Aragón, no a los documentos adjuntos en la consulta anterior. Sin embargo, no tengo acceso al texto de ese decreto legislativo en este momento, por lo que no puedo proporcionar una respuesta detallada y específica sobre su propósito principal.

Sin embargo, puedo decir que, en general, los decretos legislativos son normas jurídicas que recogen el texto articulado de una determinada materia, resultado de la integración, simplificación y sistematización de las disposiciones legales vigentes en un ámbito concreto. Por lo tanto, el Decreto Legislativo 3/2000 del Gobierno de Aragón probablemente recoge y sistematiza las disposiciones legales vigentes en una materia determinada en la comunidad autónoma de Aragón.

Para obtener una respuesta más precisa y detallada sobre el propósito del Decreto Legislativo 3/2000 del Gobierno de Aragón, sería necesario revisar el texto de ese decreto legislativo.

Referencias:
- No disponible en este momento.

In [12]:
response = qe_leg.query("¿Qué es una reforma laboral?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La expresión "reforma laboral" no está específicamente definida en los documentos del BOE (Boletín Oficial del Estado) a los que me remito. Sin embargo, en el contexto de las modificaciones sustanciales de las condiciones de trabajo, una "reforma laboral" puede incluir cambios en diversos aspectos de las condiciones de trabajo de los empleados.

Por ejemplo, la jornada laboral, tal como se define en el artículo 10 del BOE-A-2001-19309, puede ser objeto de cambios en el contexto de una reforma laboral. La jornada ordinaria de trabajo en el año 1998 era de mil setecientas noventa y nueve horas anuales de trabajo efectivo, distribuidas entre doscientos veinticinco días de los doscientos setenta y tres laborables al año, como máximo. Los trabajadores tienen derecho a un descanso semanal continuado de dos días, siendo uno de éstos domingo y pudiendo cada empresa distribuir el trabajo entre sus empleados en los seis días de la semana.

Además, el artículo 41 del BOE-A-2001-19309 menciona la posibilidad de cambiar el destino de un trabajador a un trabajo de categoría inferior, siempre y cuando se respete la retribución correspondiente al trabajo efectivamente realizado.

Por lo tanto, una "reforma laboral" puede implicar cambios en las condiciones de trabajo de los empleados, como la jornada laboral, siempre y cuando se respeten sus derechos y obligaciones establecidos en el BOE.

Referencia:
- BOE-A-2001-19309, artículos 10 y 41.

In [13]:
response = qe_leg.query("¿Se ha aprobado alguna recientemente?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La información proporcionada se refiere a un modelo de certificado de aprobación para contenedores en el marco del Convenio Aduanero sobre Contenedores de 1972. Sin embargo, el documento no especifica si se ha aprobado algún contenedor recientemente. El certificado de aprobación por modelo incluye detalles sobre la clase del contenedor, número o letras de identificación del modelo, número de identificación de los planos de construcción, entre otros. No obstante, no se menciona ninguna fecha específica de aprobación reciente en el documento.

Referencia: Apéndice 2 del Anexo 5, Modelo número 2, Convenio Aduanero sobre Contenedores, 1972, Certificado de aprobación por modelo, BOE.

### Usando el índice de educación y cultura

In [14]:
response = qe_edu.query( "¿Cómo se puede acceder a las becas de estudio para el próximo curso académico?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Para solicitar una beca de estudio para el próximo curso académico, deberás tener en cuenta lo siguiente:

1. Antigüedad: Debes tener una antigüedad mínima de un año en la fecha de cierre de admisión de solicitudes.

2. Estudios: Los estudios deben ser para la obtención de títulos oficiales de carácter universitario, superior o medio, acceso a la Universidad para mayores de 25 años, Bachillerato, o Formación Profesional Específica. También se consideran los estudios no oficiales relacionados con tu trabajo habitual, siempre que sean informados favorablemente por la Comisión de Fondos Sociales.

3. Continuidad: Debes garantizar la continuidad en los estudios.

4. Plazo de solicitud: El plazo de solicitudes de becas generalmente se abre en septiembre de cada año y finaliza 15 días después de la fecha límite marcada por el Ministerio de Educación y Ciencia. Sin embargo, debes verificar la fecha límite en el sitio web oficial del Ministerio de Educación, ya que puede variar.

5. Documentación: Al solicitar la beca, deberás acompañar la solicitud con un justificante de matrícula, indicando las asignaturas matriculadas. Si sigues estudios por la modalidad de «libre», deberás presentar el justificante en un plazo máximo de quince días después de la fecha límite marcada por el Ministerio de Educación y Ciencia.

6. Número mínimo de créditos: El número mínimo de créditos fijado en los párrafos anteriores en que el alumno debe quedar matriculado en el curso para el que solicita la beca no será exigible, por una sola vez, en el caso de los alumnos a los que, para finalizar sus estudios, les reste un número de créditos inferior a dicho número mínimo, siempre que no haya disfrutado de la condición de becario durante más años de los previstos en el apartado 13 siguiente.

Referencia: BOE-A-2004-21024, artículos 59 y 60, y disposición final tercera.

Nota: La información proporcionada se basa en el Real Decreto 1721/2007, de 21 de diciembre, y puede estar sujeta a cambios. Verifica la información más reciente en el sitio web oficial del Ministerio de Educación.

In [15]:
response = qe_edu.query( "¿Cuál es el marco actual para la financiación de proyectos culturales por parte del gobierno español, y qué criterios se utilizan para determinar la elegibilidad y la asignación de fondos?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La financiación de proyectos culturales por parte del gobierno español se rige actualmente por la Orden del BOE (Boletín Oficial del Estado) con referencia BOE-A-2014-4262. El documento no especifica si el proyecto forma parte de programas de inversión y gasto público previamente aprobados, ni si está cofinanciado por una organización internacional de financiación del desarrollo. Tampoco hay información sobre si el proyecto ha sido estudiado y rechazado por una organización internacional de financiación del desarrollo o por otro miembro del CAD.

En relación a la comprobación de los precios y de la calidad de los suministros, el apartado 63 de las Directrices del CAD para la valoración de los programas (DVP) no es aplicable en este caso, ya que la financiación de proyectos culturales por parte del gobierno español no está sujeta a estas directrices.

Los criterios de elegibilidad y asignación de fondos se basan en los siguientes puntos:

1. Eficiencia y economía: La elección entre las ofertas presentadas se realizará según estos criterios, y se debe justificar expresamente en una memoria la elección cuando no recaiga en la propuesta económica más ventajosa.

2. Bienes inventariables: Se aplicará lo dispuesto en los apartados 4 y 5 del artículo 31 de la Ley General de Subvenciones.

3. Destino de los bienes: El beneficiario deberá destinar los bienes al fin concreto para el que se concedió la financiación durante un período de cinco años para bienes inscribibles en un registro público, y de dos años para el resto de bienes.

4. Plazo de ejecución: Las actuaciones financiadas deben ejecutarse desde el 1 de enero del año de la correspondiente convocatoria y hasta un plazo máximo de dieciocho meses contados desde la fecha de resolución de la concesión.

5. Presupuesto financiable mínimo: El presupuesto financiable mínimo se regulará en la correspondiente convocatoria.

6. Importe de la financiación: La financiación a conceder será del 75 por ciento sobre el presupuesto financiable.

7. Límites de financiación pública: La financiación pública total de la inversión no podrá exceder del 75 por ciento sobre el total del presupuesto del proyecto de inversión que haya sido considerado financiable.

8. Compatibilidad: La percepción de apoyo financiero será compatible con la de cualquier otra financiación pública otorgada sobre los mismos costes financiables, siempre y cuando se respete el límite fijado en el artículo 9.

9. Régimen de concesión y características de financiación: Los apoyos financieros tendrán la forma de préstamos reembolsables con un plazo de amortización de diez años, un plazo de carencia de tres años, y un tipo de interés de aplicación que se establecerá en las correspondientes convocatorias.

El documento no proporciona información sobre si el proyecto ha sido preparado, proyectado y valorado según normas y criterios que siguen, en líneas generales, las Directrices del CAD para la valoración de los programas (DVP).

Puede encontrar más detalles en el documento BOE-A-2014-4262.

### Usando el índice de administración pública y procedimientos

In [16]:
response = qe_adm.query("¿Qué procedimientos deben seguirse para solicitar una subvención para proyectos de investigación en el ámbito de la salud pública en España?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Para solicitar una subvención para proyectos de investigación en el ámbito de la salud pública en España, de acuerdo con la información contenida en el documento del BOE proporcionado, se deben seguir los siguientes procedimientos:

1. Las entidades que pueden solicitar las ayudas se dividen en tres grupos:
   A) Entidades con finalidad investigadora, legal o estatutaria, personalidad jurídica propia, capacidad suficiente de obrar y sin inhabilitación para la obtención de ayudas o subvenciones públicas, o para contratar con el Estado u otros entes públicos.
   B) Empresas que pretendan realizar un proyecto de investigación, que tengan personalidad jurídica propia y suficiente capacidad de obrar, y no se hallen inhabilitadas para la obtención de ayudas públicas o para contratar con el Estado u otros Entes públicos.
   C) Organizaciones de/o para personas con discapacidad o personas mayores legalmente constituidas, que puedan ser usuarios o beneficiarios del resultado de los proyectos de investigación, y que participen o colaboren en la realización de estos.

2. Los proyectos a financiar deben pertenecer a las líneas de investigación determinadas en cada convocatoria y desarrollarse durante uno, dos o tres años. Si la duración se estima en dos o tres años, estarán divididos en dos o tres fases diferenciadas, correspondientes cada una a un año de ejecución. La Resolución de convocatoria priorizará las líneas de investigación recogidas en el anexo a la presente Orden.

3. La solicitud debe presentarse en un plazo de un mes, a partir del día siguiente al de la publicación de la convocatoria en el «Boletín Oficial del Estado».

4. La documentación debe presentarse en cuatro ejemplares completos y dirigirse a la Delegación del Gobierno para el Plan Nacional sobre Drogas. La solicitud se puede presentar en los Registros y oficinas previstas en el artículo 38.4 de la Ley 30/1992, de 26 de noviembre, de Régimen Jurídico de las Administraciones Públicas y del Procedimiento Administrativo Común, modificada por la Ley 4/1999, de 13 de enero.

5. La solicitud debe incluir la siguiente información:
   - Indicar si el proyecto presentado está siendo subvencionado por otro organismo público o privado, de carácter nacional o internacional.
   - Los proyectos de investigación deben respetar los principios fundamentales relativos a los derechos humanos recogidos en la Declaración de Helsinki y en el Convenio del Consejo de Europa y en la legislación española sobre bioética.
   - Adjuntar la documentación especificada en la convocatoria, que puede incluir:
     - Una memoria técnica adecuada al modelo que se establezca en la Resolución de convocatoria de ayudas.
     - La justificación de la necesidad y oportunidad del proyecto, así como la viabilidad técnica y económica del mismo.
     - La relación de los miembros del equipo de investigación, con indicación de su formación y experiencia en el campo de investigación correspondiente.
     - El currículum vitae de la persona física o jurídica representada.

References:
- BOE-A-2004-17430 (<https://www.boe.es/buscar/doc.php?id=BOE-A-2004-17430>)

In [17]:
response = qe_adm.query("¿Cuáles son los requisitos para renovar el DNI en España?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Para renovar el DNI (Documento Nacional de Identidad) en España, un ciudadano debe cumplir con los siguientes requisitos:

1. Asistencia personal: La presencia física del titular del DNI es necesaria para el proceso de renovación, ya que no es posible realizarlo a través de medios telemáticos.

2. Vencimiento o daño del soporte físico (tarjeta): Si el DNI está vencido o dañado, la renovación se realizará del mismo modo que el registro inicial (véase el apartado 3.2.3 del documento).

3. Revocación: En caso de revocación, el procedimiento será el mismo, ya sea con o sin reemplazo del soporte (véase el apartado 3.3.2).

4. Legalidad: Todos los ciudadanos españoles tienen derecho a obtener un DNI, con la obligación de hacerlo para los residentes de 14 años o más. Los ciudadanos españoles que tengan la edad legal y capacidad completa pueden usar el DNIe para la identificación electrónica y firmar documentos electrónicamente (Real Decreto 1553/2005 y Ley 59/2003).

5. Requisitos específicos para DNIe: El nuevo DNIe solo se puede solicitar durante el proceso de reemplazo del documento actual. La Dirección General de la Policía organizará y programará el reemplazo de las tarjetas DNI emitidas antes del Real Decreto 1553/2005 para los residentes en España.

6. Verificación de huellas dactilares: Si alguna de las huellas dactilares no está disponible en el sistema, el ciudadano deberá solicitar la renovación en un puesto de expedición atendido por un funcionario.

7. Renovación después de 5 años desde la identificación inicial: Si han transcurrido más de 5 años desde la identificación inicial del ciudadano, la renovación a través de los Puntos de Actualización del DNIe requerirán la presencia previa del ciudadano ante un funcionario de la Oficina de Expedición (artículo 13 de la Ley de Firma Electrónica).

8. Verificación de la información del solicitante: Toda la información recabada durante la expedición anterior debe ser verificada.

9. Comprobación de las facultades de representación: La entrega del DNI y, en su caso, los certificados de identidad y firma electrónica asociados, deberá realizarse personalmente a su titular, y cuando éste sea menor de 14 años o incapaz, se llevará a cabo en presencia de la persona que tenga encomendada la patria potestad o tutela, o persona apoderada por estas últimas.

Para obtener más información, consulte el documento BOE en: <https://www.boe.es/buscar/doc.php?id=BOE-A-2006-4841>

La respuesta anterior se basa en la información proporcionada en el documento y puede no abarcar todos los escenarios posibles o actualizaciones en las regulaciones.

En el caso de que una o más huellas dactilares no estén disponibles en el sistema, el ciudadano deberá solicitar la renovación en un puesto de expedición atendido por un funcionario, como se establece en el apartado 3.2.3 del documento.

In [18]:
response.metadata

{'48520ca8-92fb-4c0b-a5b5-fa784180ef02': {'filename': 'https://www.boe.es/buscar/doc.php?id=BOE-A-2006-4841'},
 '42329fce-9d95-4884-a9bd-4d38434685cb': {'filename': 'https://www.boe.es/buscar/doc.php?id=BOE-A-2006-4841'}}